In [ ]:
import pandas as pd
import xlsxwriter
from typing import Any

In [ ]:
def foo(week=None, month=None):
    if week is not None:
        return f"{week}"
    if month is not None:
        return f"{month}"

In [ ]:
lst_week = ["W01", "W02", "W03", "W04"]
lst_month = ["M01", "M02", "M03", "M04"]
lst_fab = ["F1", "F2", "F3", "F4", "F5", "F6"]
lst_project = ["SOC+CPE", "SOC", "CPE"]

lst_all_kpi = []
for project in lst_project:
    for fab in lst_fab:
        lst_kpi = []
        lst_kpi.append(project)
        lst_kpi.append(fab)
        for time in lst_week:
            lst_kpi.append(foo(week=time))
        for time in lst_month:
            lst_kpi.append(foo(month=time))
        lst_all_kpi.append(lst_kpi)

df = pd.DataFrame(lst_all_kpi, columns=["", "FAB"] + lst_week + lst_month)
df

In [ ]:
def set_boarder(df, start, end, workbook, worksheet):
    start_row, start_col = start
    end_row, end_col = end

    for r in range(start_row, end_row + 1):
        for c in range(start_col, end_col + 1):
            opts: dict = {"align": "center", "valign": "vcenter"}
            if r == start_row:
                opts["top"] = 1
            if r == end_row:
                opts["bottom"] = 1
            if c == start_col:
                opts["left"] = 1
            if c == end_col:
                opts["right"] = 1

            df_row, df_col = r - 1, c
            cell_value = ""
            if 0 <= df_row < len(df) and 0 <= df_col < len(df.columns):
                cell_value = df.iloc[df_row, df_col]

            border_fmt = workbook.add_format(opts)
            worksheet.write(r, c, cell_value, border_fmt)


In [ ]:
file_path = "merged_output.xlsx"

with pd.ExcelWriter(file_path, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name="KPI_SR", index=False)
    df.to_excel(writer, sheet_name="KPI_CR", index=False)

    workbook: Any = writer.book
    worksheet_SR: Any = writer.sheets["KPI_SR"]

    merge_format = writer.book.add_format({"align": "center", "valign": "vcenter", "bold": True, "border": 1})  # type: ignore
    writer.sheets["KPI_SR"].merge_range(1, 0, len(lst_fab), 0, "SOC+CPE", merge_format)
    writer.sheets["KPI_SR"].merge_range(len(lst_fab) + 1, 0, 2 * len(lst_fab), 0, "SOC", merge_format)
    writer.sheets["KPI_SR"].merge_range(2*len(lst_fab) + 1, 0, 3 * len(lst_fab), 0, "CPE", merge_format)
    
    set_boarder(df, (1,1), (6,9), workbook, worksheet_SR)